In [2]:
def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Initialize network & a base strategy
</div>

In [22]:
# Create an instance of supply chain configuration
supply_chain_config = SupplyChainConfig()
dynamics = Dynamics(supply_chain_config)

# Policy parameters
hedging_base_stock_shortfall = 0.25
hedging_base_stock_random = 0.25
hedging_mean_demand_policy = 0.1

# Creat a base policy
base_stock_policy_shortfall = StaticBaseStockPolicyShortfall(supply_chain_config, hedging_base_stock_shortfall)
base_stock_levels = np.array(base_stock_policy_shortfall.base_stock_levels)


<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Generate the data
</div>

In [ ]:
from src.supply_chain_config import SupplyChainConfig
from src.dynamics import Dynamics
from src.evaluate import Evaluate
from src.policy import (
    MeanDemandPolicy,
    StaticBaseStockPolicyRandom,
    StaticBaseStockPolicyShortfall,
    EchelonBaseStockPolicy,
    sSPolicy,
    DualIndexPolicy,
    MySmartPolicy
)
import numpy as np
import pandas as pd

def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

# 1. Initialize environment
sc = SupplyChainConfig()
dyn = Dynamics(sc)

# 2. Policy parameters
hedge_random  = 0.4
hedge_shortfall = 0.4
hedge_echelon = 0.6
hedge_nonstationary = 1
hedge_smart = 0.1
hedge_sS = 1
hedge_dual_index = 0.9


num_components = sc.num_components        # Haal het aantal componenten op
s_levels_list = [hedge_sS] * num_components
S_levels_list = [hedge_sS] * num_components

s_lower_list_DI = [hedge_dual_index] * num_components
S_upper_list_DI = [hedge_dual_index] * num_components

# 3. Instantiate all policies

policy_random  = StaticBaseStockPolicyRandom(sc, hedge_random)
policy_shortfall = StaticBaseStockPolicyShortfall(sc, hedge_shortfall)
policy_echelon = EchelonBaseStockPolicy(sc, hedge_echelon)
policy_smart = MySmartPolicy(sc, hedge_smart)

policies = [policy_random, policy_echelon, policy_shortfall, policy_smart]
num_policies = len(policies)

# 4. Simulation params
num_demand_patterns    = 100
num_trajectories       = 1
periods_per_trajectory = 200

evalr = Evaluate(sc, dyn, num_trajectories, periods_per_trajectory)

# 5. Run simulations and collect
data_parts = []
for dp in range(num_demand_patterns):
    costs = evalr.compare_policies(policies)
    # assemble base‑stock targets
    bs_arr = []
    for p in policies:
        policy_levels_to_append = None
        if hasattr(p, "base_stock_levels"):
            policy_levels_to_append = p.base_stock_levels
        elif hasattr(p, "S"):
            # Controleer of p.S de verwachte structuur heeft (lijst/array van num_components)
            if isinstance(p.S, (list, np.ndarray)) and len(p.S) == sc.num_components:
                 policy_levels_to_append = p.S
            else:
                 # Dit zou niet moeten gebeuren op basis van je klasse definities, maar voor de zekerheid
                 print(f"Error: Policy {type(p).__name__} heeft een 'S' attribuut, maar het heeft niet de verwachte structuur (lijst/array van lengte {sc.num_components}).")
        bs_arr.append(policy_levels_to_append)

    bs_arr = np.array(bs_arr)
    block = np.concatenate([
        np.arange(num_policies).reshape(-1,1),
        np.ones((num_policies,1))*dp,
        bs_arr,
        np.array(costs).reshape(-1,1)
    ], axis=1)
    data_parts.append(block)

# 6. Build DataFrame & save
cols = ["policy_id", "demand_pattern"] + [f"C{i+1}" for i in range(sc.num_components)] + ["cost"]
df = pd.DataFrame(np.vstack(data_parts), columns=cols)

# Create capacity columns
capacities = sc.capacities
num_nodes = sc.num_nodes
capacity_cols = [f"N{i+1}" for i in range(num_nodes)]
for i in range(num_nodes):
    df[capacity_cols[i]] = capacities[i]

# Create lead time columns
lead_time_cols = [f"LT{i+1}" for i in range(num_components)]
lead_times = sc.lead_times
num_components = sc.num_components
for i in range(num_components):
    df[lead_time_cols[i]] = lead_times[i]

# Create Holding Costs columns
holding_costs = sc.h
holding_cost_cols = [f"HC_C{i+1}" for i in range(num_components)]
for i in range(num_components):
    df[holding_cost_cols[i]] = holding_costs[i]

# Create backlogs penalty columns
backlog_penalties = sc.p
num_products = sc.num_products
backlog_penalty_cols = [f"BP_P{i+1}" for i in range(num_products)]
for i in range(num_products):
    df[f"BP_C{sc.num_sub_components + i + 1}"] = backlog_penalties[i]

# Create demand mean columns
agg_demand_mean = sc.aggregate_demand_mean
product_demand_means = sc.product_demand_mean # Dit is een lijst voor de producten

df["AggDemandMean"] = agg_demand_mean

# Product Demand Means
mean_demand_product_cols = [f"MeanDemand_P{i+1}" for i in range(num_products)]
for i in range(num_products):
    df[f"MeanDemand_C{sc.num_components - num_products + i + 1}"] = product_demand_means[i]

df['policy_id'] = df['policy_id'].replace({
    0.0: 'policy_random',
    1.0: 'policy_echelon',
    2.0: 'policy_shortfall',
    3.0: 'policy_smart'
})
df.to_excel("data/generated_data.allpolicies6.xlsx", index=False)

print(f"Results for {num_policies} policies across {num_demand_patterns} patterns → saved to data/generated_data_allpolicies.csv")


Total Sub-component Demand:
Component 0: Needed = 18, Available = 18
Component 1: Needed = 18, Available = 18
Component 2: Needed = 17, Available = 18
Component 3: Needed = 17, Available = 18
Component 4: Needed = 17, Available = 18
Component 5: Needed = 8, Available = 9
Component 6: Needed = 6, Available = 6
Component 7: Needed = 4, Available = 4
Component 8: Needed = 14, Available = 15
Component 9: Needed = 4, Available = 4
Component 10: Needed = 18, Available = 18

Smart Action (All Nodes): [17, 17, 17, 17, 17, 8, 6, 4, 14, 4, 17, 8, 6, 4], Hedging: [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098]

Total Sub-component Demand:
Component 0: Needed = 18, Available = 15
  Shortage of 3 for component 0!
  Reduced production of component 5 to 5
Component 1: Needed = 18, Available = 15
  Shortage of 3 for component 1!
  Reduced production of component 8 to 11
Component 2: Needed = 17, Available = 16
  Shortage of 1 for component 2!
  Reduc